In [28]:
import pandas as pd
import plotly.graph_objects as go
from src.constantes import columnas_relevantes_eph, anios_educacion_nivel_ed
import plotly.express as px

In [29]:
pd.set_option('display.max_colwidth', 3000)

In [30]:
df = pd.read_csv("datos-eph/usu_individual_T323.txt", sep=";", low_memory=False,
                 )[columnas_relevantes_eph]
# df = df[df["INTENSI"] == "2"].reset_index(drop=True) # Ocupados plenos
df = df[df["CAT_OCUP"] == 3].reset_index(drop = True) #"Obreros y empleados"

# Análisis del sesgo de la muestra

¿La tasa de no respuesta está asociado a algo?

## Mujeres vs hombres

In [31]:
no_respuesta = (df["P21"] == -9).sum()
total_respuestas = len(df["P21"])
tasa_no_respuesta_total = (no_respuesta / total_respuestas)  # Calcula la tasa de no respuesta como un porcentaje
print(f"Número de no respuestas: {no_respuesta:,.0f}\nTasa de no respuesta: {tasa_no_respuesta_total*100:.2f}%" )
tasa_no_respuesta_femenina = ((df["P21"] == -9) & (df["CH04"] == 2)).sum() / ((df["CH04"] == 2).sum())
tasa_no_respuesta_masculina = ((df["P21"] == -9) & (df["CH04"] == 1)).sum() / ((df["CH04"] == 1).sum())

Número de no respuestas: 2,568
Tasa de no respuesta: 15.13%


In [32]:
fig = go.Figure()
fig.add_trace(go.Bar(y = [tasa_no_respuesta_total, tasa_no_respuesta_femenina, tasa_no_respuesta_masculina],
                     x = ["Total", "Femenina", "Masculina"], 
                     textposition="inside", texttemplate="%{y}"
                     ))

fig.update_layout(template=None, title_text="Tasa de no respuesta por sexo", width = 500)
fig.update_yaxes(tickformat=".1%")

# Cálculo de variables

In [33]:
df = df[df["P21"] > 0] #Saco las no respuestas salariales
df = df[(df["PP3E_TOT"] > 0) & (df["PP3E_TOT"] < 999)] # No respuestas tiempo

## 1- Salario horario

In [34]:
df["Salario horario"] = df["P21"] / df["PP3E_TOT"]
df.sort_values("Salario horario", ascending=False)

,ANO4,TRIMESTRE,AGLOMERADO,CH04,CH06,CH15,CH09,CH10,CH11,CH12,CH13,CH14,NIVEL_ED,INTENSI,CAT_OCUP,PP3E_TOT,PP04A,PP04D_COD,P21,Salario horario
3779,2023,3,32,2,31,1,1,2,0,4,1,NaN,4,3.0,3,54.0,2.0,10333.0,4500000,83333.333333
5431,2023,3,23,2,35,3,1,2,0,6,1,NaN,6,2.0,3,4.0,1.0,41332.0,280000,70000.000000
4340,2023,3,4,2,23,1,1,1,2,7,2,0.0,5,1.0,3,6.0,2.0,41312.0,400000,66666.666667
10405,2023,3,91,1,45,1,1,2,0,4,1,NaN,4,2.0,3,4.0,1.0,10312.0,250000,62500.000000
3974,2023,3,31,1,55,3,1,2,0,4,1,NaN,4,2.0,3,6.0,2.0,30313.0,370000,61666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10311,2023,3,33,1,51,1,1,2,0,4,2,5.0,3,1.0,3,20.0,2.0,47313.0,5000,250.000000
8641,2023,3,33,1,62,1,1,2,0,7,2,3.0,5,3.0,3,48.0,2.0,80313.0,10000,208.333333
6129,2023,3,91,2,34,1,1,2,0,4,1,NaN,4,2.0,3,25.0,1.0,10333.0,5000,200.000000
3286,2023,3,22,1,21,1,1,2,0,2,1,NaN,2,3.0,3,48.0,2.0,82314.0,9000,187.500000


In [35]:
fig = px.box(df, y="Salario horario", points="all")
fig.update_layout(template = None, width = 500)

## 2- Años de educación

In [36]:
tasa_no_respuesta_anios_educ = df["CH14"].isna().sum() / len(df.CH14)
print(f"Tasa de No respuesta de último año aprobado: {tasa_no_respuesta_anios_educ:.2%}")

Tasa de No respuesta de último año aprobado: 65.69%


In [37]:
df["Años de educ"] = df["NIVEL_ED"].apply(lambda x: anios_educacion_nivel_ed[x])

In [38]:
fig = px.box(df, x="Años de educ", y="Salario horario", color="Años de educ")
fig.update_layout(template = None, showlegend = False, width = 500)

In [ ]:
df.to_excel("EPH_procesada.xlsx", index=False)